import the libraries this program will use:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import os

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported')

Libraries imported


Import the data CSV for the Toronto postcodes:

In [2]:
sorted_neighbourhoods_geo=pd.read_csv("sorted_neighbourhoods_geo.csv")
sorted_neighbourhoods_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


Drop unneccesary columns from the dataframe:

In [11]:
commerce_only=sorted_neighbourhoods_geo[sorted_neighbourhoods_geo['Neighbourhood'].str.contains("Commerce")]
commerce_only.drop([ "Postcode", "Borough"], axis=1, inplace=True)
commerce_only=commerce_only.reset_index()
commerce_only.drop([ "index"], axis=1, inplace=True)
commerce_only

,Neighbourhood,Latitude,Longitude
0,"Commerce Court,Victoria Hotel",43.648198,-79.379817


In [ ]:
Assign the lattitude and longitude to variables:

In [ ]:
latitude_commerce=commerce_only.loc[0, 'Latitude']
longitude_commerce=commerce_only.loc[0, 'Longitude']
print('Latitude and longitude values of the centre of the commerce district are: {} and {}'.format(latitude_commerce, longitude_commerce))

Set up the API data for the FourSquare API, and retrieve the data from FourSquare:

In [13]:
CLIENT_ID = 'XXX' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20191010' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_commerce, 
    longitude_commerce, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2b577b9da7ee001bf7dc35'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 237,
  'suggestedBounds': {'ne': {'lat': 43.65719850900001,
    'lng': -79.36740215331005},
   'sw': {'lat': 43.63919849099999, 'lng': -79.39223164668999}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '501ae947e4b0d11883b910a7',
       'name': 'Equinox Bay Street',
       'location': {'address': '199 Bay St',
        'crossStreet': 'at Commerce Court West, PATH Level',
        'lat': 43.64809974034856,
        'lng': -79.37998869

Extract the venues and categories types:

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Convert the above JSON to a Pandas dataframe:

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

Restrict the results to only coffee shops:

In [8]:
coffee_only = nearby_venues[nearby_venues['categories'].isin(['Coffee Shop'])]
coffee_only.reset_index()

,index,name,categories,lat,lng
0,4,Pilot Coffee Roasters,Coffee Shop,43.648835,-79.380936
1,16,Dineen @CommerceCourt,Coffee Shop,43.648251,-79.380127
2,22,Pilot Coffee Roasters,Coffee Shop,43.645018,-79.380415
3,25,Starbucks,Coffee Shop,43.647261,-79.378599
4,40,Starbucks,Coffee Shop,43.646731,-79.383951
5,57,Starbucks,Coffee Shop,43.650159,-79.377793
6,70,M Square Coffee Co,Coffee Shop,43.651218,-79.383555
7,78,Balzac's Coffee,Coffee Shop,43.644373,-79.383065
8,82,Versus Coffee,Coffee Shop,43.651213,-79.375236


Use Folium to display the above coordinates on a map:

In [16]:
# create map of Commerce centre Toronto using latitude and longitude values
map_commerce = folium.Map(location=[latitude_commerce, longitude_commerce], zoom_start=16)

# add markers to map
for lat, lng, name in zip(coffee_only['lat'], coffee_only['lng'], coffee_only['name']):
    label = '{}'.format(name)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_commerce)  
map_commerce